<a href="https://colab.research.google.com/github/YangHee-Min/DL_fakenews/blob/main/Fakenews_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Get imports

In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.8.*"

In [ ]:
!pip install -q tf-models-official==2.7.0

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

### Download dataset from Google Drive

In [ ]:
!pip install -q --upgrade --no-cache-dir gdown
!gdown --id 1rkaWs5jNHswVGPsmoNjbGVXdkceuiUSp

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1rkaWs5jNHswVGPsmoNjbGVXdkceuiUSp
To: /content/News _dataset.zip
100% 43.1M/43.1M [00:00<00:00, 71.2MB/s]


### Unzip file

In [ ]:
import zipfile
path_to_zip_file = '/content/News _dataset.zip'
directory_to_extract_to = 'dataset'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

## Create Datasets from CSV files

In [ ]:
# Get imports
import pandas as pd

In [ ]:


def read_data(csv_path, is_true):
  def add(title, text):
    return title + "\n" + text

  data = pd.read_csv(csv_path)
  data = data.drop('subject', axis=1).drop('date', axis=1)
  data['title_text'] = data.apply(lambda row: add(row['title'], row['text']), axis=1)
  data = data.drop('title', axis=1).drop('text', axis=1)

  output_true = 0
  if is_true:
    output_true = 1
  
  data['is_true'] = output_true
  return data

# Fake data
fake_path = "./dataset/Fake.csv"
fake_data = read_data(fake_path, is_true=False)
# fake_data.head()

# True data
true_path = "./dataset/True.csv"
true_data = read_data(true_path, is_true=True)

### Split each dataset into either train, validation or test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split the corresponding datasets
t_train, t_test = train_test_split(true_data,test_size=0.2)
t_train, t_validation = train_test_split(t_train, test_size=0.2)

f_train, f_test = train_test_split(fake_data,test_size=0.2)
f_train, f_validation = train_test_split(f_train, test_size=0.2)

In [ ]:
train_set = pd.concat([t_train, f_train])
validation_set = pd.concat([t_validation, f_validation])
test_set = pd.concat([t_test, f_test])

### Convert to tensorflow datasets

#### Convert to tf.DS from pd.Dataframe

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(dict(train_set))
val_ds = tf.data.Dataset.from_tensor_slices(dict(validation_set))
test_ds = tf.data.Dataset.from_tensor_slices(dict(test_set))

#### Transform Slices DS to Prefetch DS

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(f'Review: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label}')

title_text
is_true
